# Setup

In [0]:
#Manipulating dataframes
import pandas as pd
import numpy as np

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# System Options
import os
import warnings
warnings.filterwarnings("ignore")

# Datetime
import datetime
today = datetime.date.today().strftime('%Y%m%d')

# Uploading user data from JSON files
import json
from pandas.io.json import json_normalize

# Saving models
import pickle

# Pre-processing
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Supervised Learning
from sklearn import linear_model
from sklearn.linear_model import LinearRegression, ElasticNet, Lasso, Ridge, Lars

# Preprocessing and Model Selection
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import ElasticNetCV, RidgeCV, LassoCV, LarsCV

# Working directory
path = 'path'
today = datetime.datetime.today().strftime('%Y%m%d')

In [0]:
df = pd.read_csv(path + 'Train' + today + '.csv')
print(df['y'].value_counts())

df.head()

1    303
0    303
Name: Late, dtype: int64


,createdAt,ktp.number,current_debt,TS.score,email.bounce,email.delivered,email.open,email.reject,email.spam,email.PaymentReminder_sent,email.PaymentReminder_seen,bank.bni,bank.mastercard,bank.visa,bank.SumCredit,bank.SumDebit,Age,danaVerifiedAccount_False,danaVerifiedAccount_True,status_active,status_freeze,detail.gender_Female,detail.gender_Male,emergencyContact.type_ayah,emergencyContact.type_ibu,emergencyContact.type_istri,emergencyContact.type_saudara-kandung,emergencyContact.type_saudara-sepupu,emergencyContact.type_suami,emergencyContact.type_teman-kantor,detail.penghasilan_gol1,detail.penghasilan_gol2,detail.penghasilan_gol3,detail.penghasilan_gol4,detail.penghasilan_gol5,detail.penghasilan_gol6,detail.pendidikan_d1,detail.pendidikan_d3,detail.pendidikan_s1,detail.pendidikan_s2,...,detail.industri_industri12,detail.industri_industri13,detail.industri_industri14,detail.industri_industri15,detail.industri_industri16,detail.industri_industri17,detail.industri_industri18,detail.industri_industri19,detail.industri_industri2,detail.industri_industri20,detail.industri_industri21,detail.industri_industri22,detail.industri_industri23,detail.industri_industri24,detail.industri_industri3,detail.industri_industri6,detail.industri_industri7,detail.industri_industri8,detail.industri_industri9,defaultPayment_No Default,defaultPayment_card,defaultPayment_dana,merge.ts_both,merge.ts_left_only,merge.email_both,merge.email_left_only,merge.cards_both,merge.cards_left_only,createdAt_ToD_12am-6am,createdAt_ToD_12pm-6pm,createdAt_ToD_6am-12pm,createdAt_ToD_6pm-12am,createdAt_day_Fri,createdAt_day_Mon,createdAt_day_Sat,createdAt_day_Sun,createdAt_day_Thu,createdAt_day_Tue,createdAt_day_Wed,Late
0,0.989778,0.168240,-0.235606,1.046789,-0.107211,-0.820792,-0.706554,-0.110925,0.0,-0.574654,-0.472342,-0.381516,-0.249661,-0.328089,-0.425487,-0.410497,-1.015497,-0.298142,0.298142,0.534084,-0.534084,-0.596994,0.596994,-0.26653,2.776889,-0.490414,-0.692275,-0.337100,-0.266530,-0.391077,-0.50637,0.875080,-0.441726,-0.177705,-0.185205,-0.050572,-0.101535,-0.373544,0.974807,-0.144338,...,-0.071611,-0.225695,-0.101535,-0.277350,-0.225695,-0.19245,-0.341773,-0.243843,-0.177705,-0.308175,-0.225695,-0.225695,-0.153293,-0.332388,-0.13484,-0.332388,-0.225695,3.412501,-0.087818,-0.219382,-0.360115,0.437595,1.379748,-1.379748,0.144338,-0.144338,-0.676202,0.676202,1.395481,-0.596994,-0.672199,-0.308175,-0.38673,-0.490414,-0.403986,-0.38236,-0.425115,-0.373544,2.557042,0
1,0.902345,0.047969,-0.384335,-0.576936,-0.107211,-0.765508,-0.950949,-0.110925,0.0,-0.524032,-0.472342,-0.381516,-0.249661,-0.328089,-0.425487,-0.410497,-0.870584,-0.298142,0.298142,0.534084,-0.534084,-0.596994,0.596994,-0.26653,-0.360115,-0.490414,1.444512,-0.337100,-0.266530,-0.391077,-0.50637,-1.142752,2.263846,-0.177705,-0.185205,-0.050572,-0.101535,-0.373544,0.974807,-0.144338,...,-0.071611,-0.225695,-0.101535,-0.277350,-0.225695,-0.19245,-0.341773,-0.243843,-0.177705,-0.308175,-0.225695,-0.225695,-0.153293,-0.332388,-0.13484,-0.332388,-0.225695,3.412501,-0.087818,-0.219382,-0.360115,0.437595,-0.724770,0.724770,0.144338,-0.144338,-0.676202,0.676202,1.395481,-0.596994,-0.672199,-0.308175,-0.38673,-0.490414,-0.403986,-0.38236,2.352304,-0.373544,-0.391077,0
2,0.121466,-0.941256,-0.523187,-0.576936,-0.107211,-0.765508,-0.741467,-0.110925,0.0,-0.574654,-0.472342,-0.381516,-0.249661,-0.328089,-0.425487,-0.410497,-0.146022,-0.298142,0.298142,0.534084,-0.534084,-0.596994,0.596994,-0.26653,-0.360115,-0.490414,1.444512,-0.337100,-0.266530,-0.391077,-0.50637,0.875080,-0.441726,-0.177705,-0.185205,-0.050572,-0.101535,-0.373544,0.974807,-0.144338,...,-0.071611,-0.225695,-0.101535,3.605551,-0.225695,-0.19245,-0.341773,-0.243843,-0.177705,-0.308175,-0.225695,-0.225695,-0.153293,-0.332388,-0.13484,-0.332388,-0.225695,-0.293040,-0.087818,-0.219382,-0.360115,0.437595,-0.724770,0.724770,0.144338,-0.144338,-0.676202,0.676202,1.395481,-0.596994,-0.672199,-0.308175,-0.38673,-0.490414,-0.403986,-0.38236,2.352304,-0

# Grid Search for best parameters

In [0]:
# Fit a cross validation ridge regression on the data to determine parameter alpha

reg = RidgeCV(alphas=np.logspace(-10, -5, 10000), store_cv_values=True )
reg.fit(X,Y)
ridge_alpha = reg.alpha_
ridge_alpha

9.649358675922512e-06

In [0]:
# Fit a cross validation Lasso regression on the data to determine parameter alpha

reg = LassoCV(alphas=np.logspace(-33, -29, 1000) )
reg.fit(X,Y)
lasso_alpha = reg.alpha_
lasso_alpha

7.159041085964917e-31

In [0]:
# Fit a cross validation Elastic net regression on the data to determine parameter alpha

Alphas = {}
L1s = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
for l1ratio in L1s:
  reg = ElasticNetCV(l1_ratio=l1ratio,
                     eps=0.00001, n_alphas=100,
                     cv=5, random_state=0)
  reg.fit(X, Y)
  Alphas[str(l1ratio)] = reg.alpha_

# Find the ideal likelihood ratio
a = list(Alphas.values())
b = a.index(min(a))
enet_l1ratio = list(Alphas.keys())[list(Alphas.values()).index(min(a))]

# ...and alpha
enet_alpha = Alphas[enet_l1ratio]
enet_l1ratio = float(enet_l1ratio)
print(enet_alpha, enet_l1ratio)

0.0004846050975179952 0.8


# Cross validate models to find the best performing one

In [0]:
# Create a list with all the models we use
models = []
models.append(  ('Line', LinearRegression()  ))
models.append(  ('Rdge', Ridge(alpha=ridge_alpha)  ))
models.append(  ('Lsso', Lasso(alpha=lasso_alpha)  ))
models.append(  ('Enet', ElasticNet(alpha=enet_alpha, l1_ratio=enet_l1ratio)  ))
models.append(  ('Lars', Lars()  ))

# Determine performance metrics
Variance = {}
MSE = {}
Rsq = {}
Pre = {}

# Validation set to evaluate model
for name, model in models:
  print(name)

  # Get CV scores
  mse = cross_val_score(model, X, Y, cv=5, scoring = 'neg_mean_squared_error')
  variance = cross_val_score(model, X, Y, cv=5, scoring = 'explained_variance')
  rsq = cross_val_score(model, X, Y, cv=5, scoring = 'r2')
  pre = cross_val_score(model, X, Y.round(), cv=5, scoring = 'average_precision')
  
  # Assign them to the dictionary
  MSE[name] = [mse.mean()]
  Variance[name] = [variance.mean()]
  Rsq[name] = [rsq.mean()]
  Pre[name] = [pre.mean()]

# Check options for this CV scores
#sorted(sklearn.metrics.SCORERS.keys())

Line
Rdge
Lsso
Enet
Lars


In [0]:
# Create a dataframe with each model and its score

metrics = [MSE, Variance, Rsq, Pre]
Metrics = pd.concat([pd.DataFrame(m) for m in metrics])
Metrics = np.transpose(Metrics)
Metrics.reset_index(inplace=True)
Metrics.columns = ['Model', 'MSE','Var', 'Rsq','Precision']

Metrics

,Model,MSE,Var,Rsq,Precision
0,Line,-1.397003e+23,-3.276678e+25,-3.319355e+25,0.762554
1,Rdge,-4.855366e-03,7.662439e-01,7.649797e-01,0.998718
2,Lsso,-6.896116e-03,6.830888e-01,6.816719e-01,1.000000
3,Enet,-3.546517e-03,8.495602e-01,8.476681e-01,0.997619
4,Lars,-4.795499e+44,-1.197710e+47,-1.202521e+47,0.142424


# Vote for the best model based on cross validation results


In [0]:
# Chose the model with the most votes

best1 = np.where( Metrics.MSE == max(Metrics.MSE))[0][0]
best2 = np.where( Metrics.Var == max(Metrics.Var))[0][0]
best3 = np.where( Metrics.Rsq == max(Metrics.Rsq))[0][0]
best4 = np.where( Metrics.Precision == max(Metrics.Precision))[0][0]

votes = [best1, best2, best3, best4]
if len(set(votes)) == 4:
  best_model = votes[0]
else:
  best_model = max(set(votes), key=votes.count)
best_model = Metrics.Model[best_model]
best_model

'Enet'

In [0]:
for m in models:
  if best_model == m[0]:  # Find the name of the best model
    model = m[1]          # Assign the variable model to that object

model.fit(X,Y)

ElasticNet(alpha=0.0004846050975179952, copy_X=True, fit_intercept=True,
           l1_ratio=0.8, max_iter=1000, normalize=False, positive=False,
           precompute=False, random_state=None, selection='cyclic', tol=0.0001,
           warm_start=False)

# Save Model

In [0]:
filename = 'model_'+best_model+'_'+today+'.sav'
pickle.dump(model, open(path+'Prototype/'+filename, 'wb'))

# Testing

In [0]:
loaded_model = pickle.load(open(path+filename, 'rb'))
loaded_model

ElasticNet(alpha=0.0004846050975179952, copy_X=True, fit_intercept=True,
           l1_ratio=0.8, max_iter=1000, normalize=False, positive=False,
           precompute=False, random_state=None, selection='cyclic', tol=0.0001,
           warm_start=False)

In [0]:
X.loc[214,:].to_json('Xobs.json')

In [0]:
import json
from pandas.io.json import json_normalize

with open('Xobs.json') as json_file:
    data = json.load(json_file)

Xobs = json_normalize(data)

In [0]:
loaded_model.predict(Xobs)

array([0.94646986])